# Validation GPU Phase 4.2 - SSP-RK3

**Objectif** : Valider l'intégrateur SSP-RK3 sur GPU vs CPU

- ✅ Simulation CPU SSP-RK3 de référence 
- ✅ Simulation GPU SSP-RK3 avec kernels CUDA
- ✅ Export des résultats dans `output_gpu_phase42.zip`
- 📊 **Analyse en local** avec scripts séparés

## Instructions Kaggle

1. **Activer GPU** dans les paramètres du notebook
2. **Exécuter toutes les cellules** (Runtime → Run All)  
3. **Télécharger** `output_gpu_phase42.zip` depuis l'onglet Output
4. **Extraire localement** et utiliser les scripts d'analyse :
   - `analyze_ssprk3_precision.py` - Analyse détaillée précision CPU/GPU SSP-RK3
   - `compare_ssprk3_performance.py` - Benchmarks et ordre de convergence
   - `analyze_phase42_complete.py` - Rapport complet automatique
   - `quick_analysis_phase42.py` - Vérification rapide des résultats

📋 Voir `README_validation_phase42.md` pour le guide complet.

## 1. Setup Environnement Kaggle

In [ ]:
# Cloner le dépôt et setup
!git clone https://github.com/elonmj/Projet_tutore_ARZ.git

import os
import shutil
import numpy as np
from datetime import datetime
import zipfile
import json

# Copier les fichiers nécessaires
ITEMS_TO_COPY = ['code', 'config']
for item in ITEMS_TO_COPY:
    source_path = f'Projet_tutore_ARZ/{item}'
    if os.path.exists(source_path):
        if os.path.exists(item):
            shutil.rmtree(item)
        shutil.copytree(source_path, item)
        print(f"✅ {item} copié")

print(f"📁 Répertoire: {os.getcwd()}")

## 2. Vérification CUDA

In [ ]:
# Vérification CUDA
try:
    from numba import cuda
    if cuda.is_available():
        device = cuda.get_current_device()
        print(f"✅ GPU: {device.name}")
        print(f"   Capacité: {device.compute_capability}")
    else:
        print("❌ CUDA non disponible")
except ImportError:
    print("❌ Numba CUDA non trouvé")

## 3. Simulation CPU SSP-RK3 (Référence)

In [ ]:
# Simulation CPU SSP-RK3 de référence
print("🖥️ SIMULATION CPU SSP-RK3 (référence)")
print("="*50)

# Configuration du path Python pour imports
import sys
sys.path.insert(0, '.')

start_time = datetime.now()

try:
    from code.simulation.runner import SimulationRunner
    
    runner_cpu = SimulationRunner(
        'config/scenario_ssprk3_gpu_validation.yml',
        device='cpu',
        quiet=False
    )
    
    times_cpu, states_cpu = runner_cpu.run()
    
    # Convert to numpy arrays if they are lists
    if isinstance(times_cpu, list):
        times_cpu = np.array(times_cpu)
    if isinstance(states_cpu, list):
        states_cpu = np.array(states_cpu)
    
    end_time = datetime.now()
    duration_cpu = (end_time - start_time).total_seconds()
    
    print(f"✅ Simulation CPU SSP-RK3 terminée en {duration_cpu:.1f}s")
    print(f"   Forme résultats: times={times_cpu.shape}, states={states_cpu.shape}")
    
    # Vérifier les infos CFL
    if hasattr(runner_cpu.params, '_cfl_debug'):
        cfl_info = runner_cpu.params._cfl_debug
        print(f"   CFL final: {cfl_info.get('last_cfl', 'N/A'):.3f}")
        print(f"   dt: {cfl_info.get('last_dt_corrected', 'N/A'):.6e}s")
    
    cpu_success = True
    
except Exception as e:
    print(f"❌ Erreur CPU SSP-RK3: {e}")
    import traceback
    traceback.print_exc()
    cpu_success = False
    times_cpu, states_cpu = None, None

## 4. Simulation GPU SSP-RK3 (Test)

In [ ]:
# Simulation GPU SSP-RK3 avec kernels CUDA
print("🚀 SIMULATION GPU SSP-RK3")
print("="*50)

if cpu_success:
    start_time = datetime.now()
    
    try:
        runner_gpu = SimulationRunner(
            'config/scenario_ssprk3_gpu_validation.yml',
            device='gpu',
            quiet=True
        )
        
        times_gpu, states_gpu = runner_gpu.run()
        
        # Convert to numpy arrays if they are lists
        if isinstance(times_gpu, list):
            times_gpu = np.array(times_gpu)
        if isinstance(states_gpu, list):
            states_gpu = np.array(states_gpu)
        
        end_time = datetime.now()
        duration_gpu = (end_time - start_time).total_seconds()
        
        print(f"✅ Simulation GPU SSP-RK3 terminée en {duration_gpu:.1f}s")
        print(f"   Forme résultats: times={times_gpu.shape}, states={states_gpu.shape}")
        
        # Speedup
        if duration_cpu > 0:
            speedup = duration_cpu / duration_gpu
            print(f"   🚀 Speedup SSP-RK3: {speedup:.2f}x")
        
        # Vérifier les infos CFL
        if hasattr(runner_gpu.params, '_cfl_debug'):
            cfl_info = runner_gpu.params._cfl_debug
            print(f"   CFL final: {cfl_info.get('last_cfl', 'N/A'):.3f}")
            print(f"   dt: {cfl_info.get('last_dt_corrected', 'N/A'):.6e}s")
        
        gpu_success = True
        
    except Exception as e:
        print(f"❌ Erreur GPU SSP-RK3: {e}")
        import traceback
        traceback.print_exc()
        gpu_success = False
        times_gpu, states_gpu = None, None
else:
    print("⚠️ Simulation GPU ignorée (échec CPU)")
    gpu_success = False
    times_gpu, states_gpu = None, None

## 5. Comparaison Rapide SSP-RK3

In [ ]:
# Comparaison rapide CPU vs GPU SSP-RK3
if cpu_success and gpu_success:
    print("🔍 COMPARAISON RAPIDE CPU vs GPU SSP-RK3")
    print("="*50)
    
    # Vérifier compatibilité des formes
    if times_cpu.shape == times_gpu.shape and states_cpu.shape == states_gpu.shape:
        # Calcul erreurs
        diff_states = np.abs(states_cpu - states_gpu)
        
        error_max = np.max(diff_states)
        error_mean = np.mean(diff_states)
        
        print(f"📊 Erreur maximale SSP-RK3: {error_max:.3e}")
        print(f"📊 Erreur moyenne SSP-RK3: {error_mean:.3e}")
        
        # Évaluation spécifique SSP-RK3
        if error_max < 1e-12:
            print("🟢 EXCELLENT: Précision SSP-RK3 < 1e-12")
            status = "EXCELLENT"
        elif error_max < 1e-10:
            print("🟢 TRÈS BON: Précision SSP-RK3 < 1e-10")
            status = "TRÈS BON"
        elif error_max < 1e-8:
            print("🟡 ACCEPTABLE: Précision SSP-RK3 < 1e-8")
            status = "ACCEPTABLE"
        else:
            print("🔴 PROBLÉMATIQUE: Précision SSP-RK3 > 1e-8")
            status = "PROBLÉMATIQUE"
        
        # Comparaison avec Phase 4.1 (WENO5+SSP-RK3)
        print(f"\n📈 vs PHASE 4.1 (WENO5+SSP-RK3):")
        print(f"   Phase 4.1: ~8.9e-3 (WENO5 + SSP-RK3)")
        print(f"   Phase 4.2: {error_max:.3e} (First-order + SSP-RK3)")
        print(f"   Note: Différence due au schéma spatial (WENO5 vs 1er ordre)")
        
        comparison_success = True
        
    else:
        print("❌ Formes incompatibles CPU/GPU")
        print(f"   CPU: times={times_cpu.shape}, states={states_cpu.shape}")
        print(f"   GPU: times={times_gpu.shape}, states={states_gpu.shape}")
        comparison_success = False
        error_max, error_mean, status = None, None, "ÉCHEC"
        
else:
    print("⚠️ Comparaison impossible (échec simulation)")
    comparison_success = False
    error_max, error_mean, status = None, None, "ÉCHEC"

## 6. Export des Résultats Phase 4.2

In [ ]:
# Export des résultats SSP-RK3 pour analyse locale
print("📦 EXPORT DES RÉSULTATS PHASE 4.2")
print("="*40)

# Créer dossier output spécifique Phase 4.2
output_dir = "output_gpu_phase42"
os.makedirs(output_dir, exist_ok=True)

# Timestamp pour les fichiers
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

files_exported = []

# Sauvegarder résultats CPU SSP-RK3
if cpu_success:
    cpu_file = f"{output_dir}/results_cpu_ssprk3_{timestamp}.npz"
    np.savez_compressed(cpu_file, 
                       times=times_cpu, 
                       states=states_cpu,
                       grid_info=runner_cpu.grid,
                       params_info=runner_cpu.params)
    files_exported.append(cpu_file)
    print(f"✅ CPU SSP-RK3: {cpu_file}")

# Sauvegarder résultats GPU SSP-RK3
if gpu_success:
    gpu_file = f"{output_dir}/results_gpu_ssprk3_{timestamp}.npz"
    np.savez_compressed(gpu_file, 
                       times=times_gpu, 
                       states=states_gpu,
                       grid_info=runner_gpu.grid,
                       params_info=runner_gpu.params)
    files_exported.append(gpu_file)
    print(f"✅ GPU SSP-RK3: {gpu_file}")

# Métadonnées de la validation Phase 4.2
metadata = {
    "timestamp": timestamp,
    "cpu_success": cpu_success,
    "gpu_success": gpu_success,
    "comparison_success": comparison_success,
    "cpu_duration": duration_cpu if cpu_success else None,
    "gpu_duration": duration_gpu if gpu_success else None,
    "speedup": duration_cpu/duration_gpu if cpu_success and gpu_success else None,
    "error_max": float(error_max) if error_max is not None else None,
    "error_mean": float(error_mean) if error_mean is not None else None,
    "status": status,
    "phase": "4.2",
    "time_integrator": "ssprk3",
    "spatial_scheme": "first_order",
    "objective": "< 1e-10"
}

metadata_file = f"{output_dir}/validation_metadata_phase42_{timestamp}.json"
with open(metadata_file, 'w') as f:
    json.dump(metadata, f, indent=2)
files_exported.append(metadata_file)
print(f"✅ Métadonnées: {metadata_file}")

# Informations de configuration SSP-RK3
config_info = {
    "scenario": "scenario_ssprk3_gpu_validation.yml",
    "grid": {"N": 200, "xmin": 0.0, "xmax": 1000.0, "dx": 5.0},
    "temporal": {"t_final": 10.0, "output_dt": 1.0},
    "numerics": {"cfl_number": 0.4, "spatial_scheme": "first_order", "time_scheme": "ssprk3"},
    "phase": "4.2",
    "focus": "SSP-RK3 GPU validation"
}

config_file = f"{output_dir}/config_info_phase42_{timestamp}.json"
with open(config_file, 'w') as f:
    json.dump(config_info, f, indent=2)
files_exported.append(config_file)
print(f"✅ Config: {config_file}")

print(f"\n📁 Fichiers dans {output_dir}:")
for file in files_exported:
    size = os.path.getsize(file) / 1024  # KB
    print(f"   {file} ({size:.1f} KB)")

## 7. Création ZIP Final Phase 4.2

In [ ]:
# Créer le ZIP final Phase 4.2 (nom fixe)
zip_filename = "output_gpu_phase42.zip"

with zipfile.ZipFile(zip_filename, 'w', zipfile.ZIP_DEFLATED) as zipf:
    for file in files_exported:
        zipf.write(file, os.path.basename(file))

zip_size = os.path.getsize(zip_filename) / (1024*1024)  # MB

print(f"📦 ZIP créé: {zip_filename} ({zip_size:.2f} MB)")
print(f"\n📥 INSTRUCTIONS:")
print(f"1. Télécharger {zip_filename} depuis l'onglet Output")
print(f"2. Extraire en local")
print(f"3. Utiliser les scripts d'analyse locaux :")
print(f"   - analyze_ssprk3_precision.py")
print(f"   - compare_ssprk3_performance.py") 
print(f"   - analyze_phase42_complete.py")

# Résumé final Phase 4.2
print(f"\n🎯 RÉSUMÉ VALIDATION PHASE 4.2")
print(f"="*50)
print(f"Simulation CPU SSP-RK3: {'✅' if cpu_success else '❌'}")
print(f"Simulation GPU SSP-RK3: {'✅' if gpu_success else '❌'}")
print(f"Comparaison: {'✅' if comparison_success else '❌'}")
if comparison_success:
    print(f"Statut: {status}")
    print(f"Erreur max: {error_max:.3e}")
    if gpu_success and cpu_success:
        print(f"Speedup SSP-RK3: {duration_cpu/duration_gpu:.2f}x")
print(f"Export: ✅ {zip_filename}")

print(f"\n✅ VALIDATION PHASE 4.2 TERMINÉE")
print(f"🚀 Prêt pour analyse SSP-RK3 en local")

In [ ]:
# Validation GPU Phase 4.2 - SSP-RK3

**Objectif** : Valider l'intégrateur SSP-RK3 sur GPU vs CPU

- ✅ Simulation CPU SSP-RK3 de référence
- ✅ Simulation GPU SSP-RK3 avec kernels CUDA
- ✅ Export des résultats dans `output_gpu_phase42.zip`
- 📊 **Analyse en local** avec scripts séparés

## Instructions Kaggle

1. **Activer GPU** dans les paramètres du notebook
2. **Exécuter toutes les cellules** (Runtime → Run All)
3. **Télécharger** `output_gpu_phase42.zip` depuis l'onglet Output
4. **Extraire localement** et utiliser les scripts d'analyse :
   - `analyze_ssprk3_precision.py` - Analyse détaillée précision CPU/GPU SSP-RK3
   - `compare_ssprk3_performance.py` - Benchmarks et ordre de convergence
   - `analyze_phase42_complete.py` - Rapport complet automatique
   - `quick_analysis_phase42.py` - Vérification rapide des résultats

📋 Voir `README_validation_phase42.md` pour le guide complet.